In [1]:
#%pip install git+https://github.com/royerlab/tracksdata
%pip install git+https://github.com/yfukai/tracksdata@fix_sqlgraph_attr 

  Cloning https://github.com/yfukai/tracksdata (to revision fix_sqlgraph_attr) to /private/var/folders/81/kh0zpmbd00g39vwz69lkk4mc0000gn/T/pip-req-build-s1a4qz40
  Running command git clone --filter=blob:none --quiet https://github.com/yfukai/tracksdata /private/var/folders/81/kh0zpmbd00g39vwz69lkk4mc0000gn/T/pip-req-build-s1a4qz40
  Running command git checkout -b fix_sqlgraph_attr --track origin/fix_sqlgraph_attr
  Switched to a new branch 'fix_sqlgraph_attr'
  Branch 'fix_sqlgraph_attr' set up to track remote branch 'fix_sqlgraph_attr' from 'origin'.
  Resolved https://github.com/yfukai/tracksdata to commit 59d6db278e270a2c2680eb42cec425ae3136257e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
image_path = "/Volumes/work/fukai/241203-cell-picker/aligned_image.zarr"
graph_path = "/Users/fukai/Downloads/aligned_labels_tracked3.db"

In [3]:
import tracksdata as td
import os
os.environ["NAPARI_ASYNC"] = "1"
import zarr
from dask import array as da

In [4]:
image = da.from_zarr(zarr.open(image_path, mode="r"))

In [5]:
graph = td.graph.SQLGraph(drivername="sqlite", database=graph_path)

In [6]:
#gav = td.array.GraphArrayView(graph, shape=tuple([image.shape[0], *image.shape[2:]]), attr_key="label")

In [7]:
from napari import Viewer
import napari_travali2 as travali2
from napari_travali2._logging import logger
from napari_travali2.actionable_tracks import actionable_tracks as at

In [ ]:
tracks = at.ActionableTracks(graph, tracklet_id_attr_name="label")

In [9]:
gav = td.array.GraphArrayView(tracks.graph, shape=tuple([image.shape[0],*image.shape[2:]]), attr_key="label")

In [ ]:
import importlib
importlib.reload(travali2)
importlib.reload(travali2._stateful_widget)
import numpy as np

logger.setLevel("DEBUG")
logger.info("Starting napari-travali2")

viewer = Viewer()

widget = travali2.StateMachineWidget(viewer, tracks, image[:,0,:,:], 
                            verified_track_ids=[],
                            candidate_track_ids=[],
                            crop_size=2048,
                            gav=gav)
viewer.window.add_dock_widget(widget, area="right")
viewer.dims.set_current_step(0,0)

INFO:napari_travali2._logging:Starting napari-travali2


INFO:napari_travali2._logging:StateMachineWidget initialized.
INFO:napari_travali2._logging:Image dtype: uint8, Label dtype: <class 'numpy.int64'>
INFO:napari_travali2._logging:Image shape: (1940, 20805, 14617), GAV shape: (1940, 20805, 14617)
INFO:napari_travali2._logging:Cropped shape: (1940, 2048, 2048)
INFO:napari_travali2._logging:Time-like shape: (1940,), Space-like shape: (20805, 14617)
INFO:napari_travali2._logging:Layer 'image' set to visible.
INFO:napari_travali2._logging:Layer 'labels' set to visible.
INFO:napari_travali2._logging:Layer 'cropped_image' set to invisible.
INFO:napari_travali2._logging:Layer 'cropped_labels' set to invisible.
INFO:napari_travali2._logging:Layer 'redraw' set to invisible.


/Users/fukai/projects/napari-travali2/.venv/lib/python3.12/site-packages/napari/_vispy/layers/scalar_field.py:198: UserWarning: data shape (20805, 14617) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
INFO:napari_travali2._logging:Region clicked
INFO:napari_travali2._logging:clicked at [    0 11767  4523]
INFO:napari_travali2._logging:Region selected: coords [    0 11767  4523]
INFO:napari_travali2._logging:window selected: (slice(None, None, None), slice(np.int64(10743), np.int64(12791), None), slice(np.int64(3499), np.int64(5547), None))
INFO:napari_travali2._logging:Layer 'image' set to invisible.
INFO:napari_travali2._logging:Layer 'labels' set to invisible.
INFO:napari_travali2._logging:Layer 'cropped_image' set to visible.
INFO:napari_travali2._logging:Layer 'cropped_labels' set to visible.
INFO:napari_travali2._logging:Layer 'redraw' set to invisible.
INFO:napari_travali2._logging:()
INFO:napari

In [13]:
widget._cropped_labels_layer.refresh()

In [14]:
gav2 = td.array.GraphArrayView(tracks.graph, shape=tuple([image.shape[0],*image.shape[2:]]), attr_key="label")

In [15]:
viewer.add_labels(gav2[:1], name="labels2")

<Labels layer 'labels2' at 0xc21b0a540>

/Users/fukai/projects/napari-travali2/.venv/lib/python3.12/site-packages/napari/_vispy/layers/scalar_field.py:198: UserWarning: data shape (20805, 14617) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(


In [33]:
from os import path
LOGGING_PATH = ".napari-travali/log.txt"
import logging 
import numpy as np
import polars as pl

SELECTED_COLOR = (1, 0, 0, 1)
DAUGHTER_COLORS = [
    (0, 1, 0, 1),
    (0, 0, 1, 1)
]
logging_path = path.join(path.expanduser("~"), LOGGING_PATH)
os.makedirs(path.dirname(logging_path), exist_ok=True)

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)  # Set the logger level to DEBUG
file_handler = logging.FileHandler(logging_path)
file_handler.setLevel(logging.DEBUG)  # Set the handler level to DEBUG
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)
logger.propagate = False  # Prevent propagation to the root logger
logger.info("Track clicked")


def find_track_successors(graph, bbox_spatial_filter, data_coordinates) -> tuple[int, pl.DataFrame]:
    logger.debug(f"data_coordinates: {data_coordinates}")
    nodes_df = bbox_spatial_filter[tuple([
        slice(c,c) for c in data_coordinates
    ])].node_attrs(attr_keys=["node_id", "label"])
    logger.debug("node_df built.")
    if nodes_df.is_empty():
        logger.info("No nodes in the bbox.")
        return -1, pl.DataFrame()
    track_id = nodes_df["label"].to_list()[0]
    # TODO make the following faster
    sorted_node_ids = graph.filter(td.NodeAttr("label") == track_id).node_attrs(attr_keys=["node_id","t"]).sort("t")["node_id"]
    logger.debug("sorted_node_ids built.")
    last_node_id = sorted_node_ids.last()
    #successors_df = graph.successors(last_node_id, return_attrs=True, attr_keys=["track_id"])
    successors_df = graph.successors(last_node_id, attr_keys=["label"])
    logger.debug("successors_df built.")
    return track_id, successors_df

def track_clicked(viewer, event):
    logger.info(event.modifiers)
    logger.info("Track clicked")
    yield  # important to avoid a potential bug when selecting the daughter
    logger.info("button released")
    data_coordinates = layer.world_to_data(event.position)
    logger.debug(f"world coordinates: {event.position}")
    logger.debug(f"data coordinates: {data_coordinates}")
    track_id, successors_df = find_track_successors(graph, gav._spatial_filter, data_coordinates)
    if track_id == -1:
        logger.info("No track found.")
        return
    if len(successors_df) == 0:
        logger.info("No successors found.")
        successor_track_ids = []
    else:
        successor_track_ids = successors_df["track_id"].to_list()
    # get the row of nodes_df (polars dataframe) with the selected track_id and largest t
    #first_node_id = sorted_node_ids.first()
    logger.info(f"selected track_id: {track_id}, "
                f"successor track_ids: {successor_track_ids}.")
    layer.colormap = {0:(0,0,0,0), 
                    track_id:SELECTED_COLOR,
                **{d:c for d,c in zip(successor_track_ids,DAUGHTER_COLORS)},
                    None:(0,0,0,0)}
    logger.info("colormap updated.")
#    selected_graph = graph.filter(td.NodeAttr("track_id").is_in([track_id, *successor_track_ids])).subgraph()
#    graph_view = td.array.GraphArrayView(selected_graph, labels.shape, attr_key="track_id")
#    selected_layer.data = graph_view
    
    
viewer.mouse_drag_callbacks.clear()
viewer.mouse_drag_callbacks.append(track_clicked)

In [ ]:
from napari.utils.colormaps import label_colormap
layer.colormap = label_colormap(
            49, 0, background_value=0
    )
layer.colormap = {0:(0,0,0,0), None:(0,0,0,0)}
track_id = 181254
layer.colormap = {0:(0,0,0,0), 
                track_id:SELECTED_COLOR,
                None:(0,0,0,0)}